## Purpose

This notebook implements the DaNN.

In [2]:
import tensorflow as tf
import math
from keras.datasets import mnist
from keras.utils import to_categorical
from keras import layers
from keras import models
from keras.models import Model
from keras import backend as K
from keras.regularizers import Regularizer
import warnings
warnings.filterwarnings('ignore')
import cv2
import numpy as np
import h5py

## Data

##### Let us begin by loading and preprocessing the source data: MNIST

In [3]:
(source_train_images, source_train_labels), (source_test_images, source_test_labels) = mnist.load_data()

In [4]:
#source_train_images = source_train_images.reshape((60000, 28, 28, 1))
source_train_images = source_train_images.reshape((60000, 28*28))
source_train_images = source_train_images.astype('float32') / 255

#source_test_images = source_test_images.reshape((10000, 28, 28, 1))
source_test_images = source_test_images.reshape((10000, 28*28))
source_test_images = source_test_images.astype('float32') / 255

source_train_labels = to_categorical(source_train_labels)
source_test_labels = to_categorical(source_test_labels)

##### Let us now load and preprocess the target data: USPS

In [5]:
with h5py.File('usps_dataset.h5', 'r') as hf:
        train = hf.get('train')
        target_train_images_aux = train.get('data')[:]
        target_train_labels = train.get('target')[:]
        test = hf.get('test')
        target_test_images_aux = test.get('data')[:]
        target_test_labels = test.get('target')[:]

In [6]:
target_train_images = []
target_test_images = []

for i in range(7291):
    img = target_train_images_aux[i].reshape(16,16)
    img = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_CUBIC)
    target_train_images.append(img.flatten())
    
for i in range(2007):
    img = target_test_images_aux[i].reshape(16,16)
    img = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_CUBIC)
    target_test_images.append(img.flatten())

In [7]:
#convert list to numpy arrays
target_train_images = np.asarray(target_train_images)
target_test_images = np.asarray(target_test_images)

#train images
#target_train_images = target_train_images.reshape((7291, 28, 28, 1))
target_train_images = target_train_images.astype('float32')
for i in range(7291):
    min_aux = min(target_train_images[i])
    max_aux = max(target_train_images[i]-min_aux)
    target_train_images[i] = (target_train_images[i]-min_aux)/max_aux

#test images
#target_test_images = target_test_images.reshape((2007, 28, 28, 1))
target_test_images = target_test_images.astype('float32')
for i in range(2007):
    min_aux = min(target_test_images[i])
    max_aux = max(target_test_images[i]-min_aux)
    target_test_images[i] = (target_test_images[i]-min_aux)/max_aux

#labels
target_train_labels = to_categorical(target_train_labels)
target_test_labels = to_categorical(target_test_labels)

## Loss

Let us now code the $MMD^2_e(\boldsymbol{q}_s, \boldsymbol{\bar{q}}_t)$ loss where
$$
\boldsymbol{q}_s=W_1^Tx_s+b
$$ 
and 
$$
\boldsymbol{\bar{q}}_t = W_1^Tx_t+b.
$$ 

In [8]:
def compute_pairwise_distances(x, y):
    
    if not len(x.get_shape()) == len(y.get_shape()) == 2:
        raise ValueError('Both inputs should be matrices.')

    if x.get_shape().as_list()[1] != y.get_shape().as_list()[1]:
        raise ValueError('The number of features should be the same.')

    norm = lambda x: tf.reduce_sum(tf.square(x), 1)
    
    return tf.transpose(norm(tf.expand_dims(x, 2) - tf.transpose(y)))

In [9]:
def gaussian_kernel_matrix(x, y, sigmas = tf.constant([1e-2, 1e-1, 1, 5, 10])):
    
    beta = 1. / (2. * (tf.expand_dims(sigmas, 1)))
    dist = compute_pairwise_distances(x, y)
    s = tf.matmul(beta, tf.reshape(dist, (1, -1)))
    
    return tf.reshape(tf.reduce_sum(tf.exp(-s), 0), tf.shape(dist))

In [10]:
#loss
def mmd(y_true, y_pred):
    
    #revert the concatenation so as to recover the source and target outputs of the hidden layer
    y_pred_source = y_pred[:, :256]
    y_pred_target = y_pred[:, 256:]
    
    kernel = gaussian_kernel_matrix
    cost = tf.reduce_mean(kernel(y_pred_source, y_pred_source))
    cost += tf.reduce_mean(kernel(y_pred_target, y_pred_target))
    cost -= 2 * tf.reduce_mean(kernel(y_pred_source, y_pred_target))
    
    #cost has to be non-negative
    cost = tf.where(cost > 0, cost, 0)
    
    return cost

## Model

The model consists of a single hidden layer with 256 nodes.
* Inputs: the source and target inputs are paralelly fed into the model (i.e, the model does not change weights).
* Outputs: the network has two outputs. The first one is the prediction of the source data and the second one is the concatenation of the outputs of the source and target data by the hidden layer.

In [11]:
input_s = layers.Input(shape=(28*28,), name='source_input')
input_t = layers.Input(shape=(28*28,), name='target_input')
hidden = layers.Dense(256, activation='softplus', name='hidden')
#prediction = layers.Dense(10, activation='softmax', name='pred')

hidden_s = hidden(input_s)
hidden_t = hidden(input_t)
aux_output = layers.concatenate([hidden_s, hidden_t], name='aux_output')
pred_s = layers.Dense(10, activation='softmax', name='source_output')(hidden_s)
#pred_s = prediction(hidden_s)
#pred_t = prediction(hidden_t)

model = Model([input_s, input_t], [pred_s, aux_output])

In [12]:
mmd_weight = 1.
model.compile(loss=['categorical_crossentropy', mmd],
              loss_weights=[1., mmd_weight], 
              optimizer='sgd', 
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
model.fit({'source_input': source_train_images, 'target_input': target_train_images},
          {'source_output': source_train_labels, 'aux_output': np.zeros(1)},
          epochs=5, batch_size=64)


ValueError: All input arrays (x) should have the same number of samples. Got array shapes: [(60000, 784), (7291, 784)]